In [1]:
%%html
<!-- definir quelques styles custom pour l'ensemble du notebook -->
<style>    
    @import url("css/custom_styles.css")
</style>

<center>
    <h1>
    Transformation Et Manipulation Des Données<br>
    </h1>
    MovieLens - Système de recommendations de films par regroupement<br>
    <br>
    <b>Jean-Francois Gagnon</b><br>
    <br>
    420-A56<br>
    <br>
</center>

# Introduction
<br>
<font class="answer">
    
J'ai choisi la base de données [MovieLens Small](https://tinyurl.com/bdhmcfht). Elle se compose de 100836 notes représentant 9742 films. Elle contient également les métadonnées sur 19 genres et des liens sur [TMDB](https://www.themoviedb.org) qui permettront d'augmemter son contenu.
    
Plus spécifiquement, (Voir https://files.grouplens.org/datasets/movielens/ml-latest-small-README.html et décrire en détails le format)
    
    
    
L'objectif de ce projet est de contruire un système de recommentation de films en utilisant les techniques de regroupement vu dans le cours. Les détails du sytème seront abordés plus loins dans ce notebooks

In [2]:
#
# imports utilitaires
#

%matplotlib inline

import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import requests
import seaborn as sns
import time

from bs4 import BeautifulSoup
from imblearn.under_sampling import RandomUnderSampler
from sklearn.cluster import MiniBatchKMeans
from sklearn.decomposition import TruncatedSVD

from tqdm.notebook import tqdm

#
# imports faisant partie de mes propres module
#

import helpers as hlp
import helpers.dataset.MovieLens as mvl
import helpers.WebScraping as scrap
import helpers.Clustering as clstr

from helpers.jupyter import display_html

# Prétraitement

<font class="answer">

Description ici?

In [3]:
#
# parametres configurant nos traitemens
#
configs = hlp.get_configs("config_overrides.json")

#
# obtenir le dataset
#
mvl_dataset = mvl.load(configs.dataset)

## links.csv

<font class="answer">
    
links.csv n'est pas utilisé directement pour le clustering. Cependant, il le sera pour complémenter l'information des autres base de données. Il m'apparait donc imporant de faire un survol rapide.

In [4]:
print("Links", mvl_dataset.links.shape)
print("Head")
display(mvl_dataset.links.head())
clstr.show_na(mvl_dataset.links)
clstr.show_types(mvl_dataset.links)

Links (9742, 3)
Head


,movieId,imdbId,tmdbId
0,1,0114709,862.0
1,2,0113497,8844.0
2,3,0113228,15602.0
3,4,0114885,31357.0
4,5,0113041,11862.0


Valeur manquante 8 (0.1%)


,movieId,imdbId,tmdbId
624,791,0113610,NaN
843,1107,0102336,NaN
2141,2851,0081454,NaN
3027,4051,0056600,NaN
5532,26587,0092337,NaN
5854,32600,0377059,NaN
6059,40697,0105946,NaN
7382,79299,0874957,NaN


Types


,movieId,imdbId,tmdbId
Type,int64,string[python],float64


<font class="answer">
    
Il manque quelques liens sur [TMDB](https://www.themoviedb.org/) (ce qui explique le type float64). Les liens [IMDB](https://www.imdb.com/) seront par conséquent privélégiés pour fin de web scrapping.

## movies.csv

<font class="answer">
    
Mettre description ici?

In [5]:
print("Movies", mvl_dataset.movies.shape)
print("Head")
display(mvl_dataset.movies.head())
clstr.show_na(mvl_dataset.movies)
clstr.show_types(mvl_dataset.movies)

Movies (9742, 3)
Head


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


Valeur manquante 0 (0.0%)
Types


,movieId,title,genres
Type,int64,object,object


<font class="answer">

Tel que décris par MovieLens, *title* contient l'année de parution. Nous allons l'extraire afin d'obtenir une nouvelle variable. *title* sera adressé un pleu plus bas.

In [6]:
def split_title_year(title, regex):
    x = regex.search(title)
    if x:
        title = x.group(1)
        year = x.group(2)
        year = int(year) if year else pd.NA
    else:
        year = pd.NA
    
    return pd.Series({"title": title, "year": year})

title_year_re = re.compile(configs.dataset.title_regex, flags=0)        
title_year = mvl_dataset.movies.title.apply(split_title_year, args=(title_year_re,))

#
# validation de l'extraction
#
print("Informations extraites", title_year.shape)
print("Head")
display(title_year.head())
clstr.show_na(title_year)
clstr.show_types(title_year)

Informations extraites (9742, 2)
Head


,title,year
0,Toy Story,1995
1,Jumanji,1995
2,Grumpier Old Men,1995
3,Waiting to Exhale,1995
4,Father of the Bride Part II,1995


Valeur manquante 12 (0.1%)


,title,year
6059,Babylon 5,<NA>
9031,Ready Player One,<NA>
9091,Hyena Road,<NA>
9138,The Adventures of Sherlock Holmes and Doctor W...,<NA>
9179,Nocturnal Animals,<NA>
9259,Paterson,<NA>
9367,Moonlight,<NA>
9448,The OA,<NA>
9514,Cosmos,<NA>
9515,Maria Bamford: Old Baby,<NA>


Types


,title,year
Type,object,object


<font class="answer">
   
On peut remarquer qu'il manque quelques années de parution. Il est probablement possible de les fixer en utilisant *links.imdbId* avec le web scrapping.

In [7]:
def imdb_scap_year(response, index, imdbId):
    success = False
    if response.ok:
        bs = BeautifulSoup(response.text, "html.parser")

        imdb_info = bs.find("script", attrs={"type": "application/ld+json"})
        imdb_json = json.loads(imdb_info.text)
        date_published = scrap.get_nested_property(imdb_json, ["datePublished"])
        if not success and date_published:
            date_published = pd.to_datetime(date_published)
            year = date_published.year
            success = True

        imdb_info = bs.find("script", attrs={"type": "application/json"})
        imdb_json = json.loads(imdb_info.text)
        releaseYear = scrap.get_nested_property(imdb_json, 
                                                ["props", 
                                                 "pageProps", 
                                                 "mainColumnData", 
                                                 "releaseYear",
                                                 "year"])
        if not success and releaseYear:
            year = releaseYear
            success = True

    if success:
        return (index, year, response.url)
    else:
        return (index, None, response.url)

def imdb_scap_year_apply_results(final_results):
    for index, year, url in final_results:
        if year is None:
            print(url, title_year.title[index], "Failed")
        else:
            title_year.year[index] = year

#    
# corriger year via web scrapping avec imdb
#
year_na = title_year.year.isna()
year_link = mvl_dataset.links.imdbId[ year_na ]

if year_link.shape[0] > 0:
    with hlp.Profile() as year_profile:
        results = scrap.imdb_requests_parallel(year_link,
                                               configs.web_scraping,
                                               imdb_scap_year,
                                               executor=configs.executor)
        imdb_scap_year_apply_results(results)
    print(f"Web scraping year: {year_profile.round_duration(2)}s")

# validation du scapping
clstr.show_na(title_year);

  0%|          | 0/12 [00:00<?, ?it/s]

Web scraping year: 2.18s
Valeur manquante 0 (0.0%)


In [8]:
#
# mettre a jour movies
#
mvl_dataset.movies["year"] = title_year.year.astype(np.int64)

In [9]:
#
# validation effet extraction de l'annee de title
#
imdb_ids = mvl_dataset.links.imdbId[mvl_dataset.links.movieId == mvl_dataset.movies.movieId]

title = mvl_dataset.movies[["movieId", "year"]].copy()
title["title"] = title_year.title.copy()
title["imdbId"] = imdb_ids.copy()

def title_agregate(dataframe):
    return pd.Series({"imdbIds": dataframe.imdbId.unique(),
                      "years": dataframe.year.unique(),
                      "counts": dataframe.movieId.nunique()})

groups = title.groupby("title").apply(title_agregate)
groups.sort_values(by="counts", ascending=False, inplace=True)

display(groups.head())

,imdbIds,years,counts
title,,,
Hamlet,"[0116477, 0040416, 0058175, 0171359, 0099726]","[1996, 1948, 1964, 2000, 1990]",5
"Christmas Carol, A","[0029992, 1067106, 0216621, 0188504]","[1938, 2009, 1999, 1977]",4
"Misérables, Les","[0113828, 0119683, 1707386, 0230534]","[1995, 1998, 2012, 2000]",4
Jane Eyre,"[0116684, 0036969, 0065911, 1229822]","[1996, 1944, 1970, 2011]",4
"Three Musketeers, The","[0108333, 0040876, 0072281, 1509767]","[1993, 1948, 1973, 2011]",4


<font class="answer">
    
Une inspection manuelle de [Hamlet 1996](http://www.imdb.com/title/tt0116477) et [Hamlet 1948](http://www.imdb.com/title/tt0040416) via IMDB permet de voir que l'année juxtaposée au titre est un identifiant unique. Je dois donc garder *title* original.

In [10]:
#
# validation doublons
#
print("Vérifier doublons")
print("Avant:", mvl_dataset.movies.shape)
mvl_dataset.movies.drop_duplicates(inplace=True)
print("Après:", mvl_dataset.movies.shape)

Vérifier doublons
Avant: (9742, 4)
Après: (9742, 4)


<font class="answer">
Aucun doublon

In [11]:
#
# validation modalites genres
#
def genres_str_to_array(genres, splitter):
    if isinstance(genres, str):
        return genres.split(splitter)
    else:
        return genres

def gather_unique_genres(genres_str, genres_splitter, final_set):
    genres_array = genres_str_to_array(genres_str, genres_splitter)
    if len(genres_array) > 0:
        final_set.update(genres_array)
    
def show_unique_genres():
    genres_set = set()
    mvl_dataset.movies.genres.apply(gather_unique_genres,
                                    args=(configs.dataset.genre_splitter, genres_set))
    
    print("Genres:", len(genres_set), "modalités")
    display(genres_set)
    
show_unique_genres()

Genres: 20 modalités


{'(no genres listed)',
 'Action',
 'Adventure',
 'Animation',
 'Children',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'IMAX',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western'}

<font class="answer">

Une seule modalité semble étrange: IMAX. En effet, elle représente plus un médium qu'un genre. Aussi, (no genres listed) semble être en fait des valeurs manquantes. Adressé un peu plus loin dans le notebook.

In [12]:
#
# inspecter comment IMAX est utilise
#
def list_size(genres_str, splitter):
    return len( genres_str.split(splitter) )

imax_crit = mvl_dataset.movies.genres.str.contains("IMAX", regex=False)
imax = mvl_dataset.movies[imax_crit].copy()
imax["genres_count"] = imax.genres.apply(list_size, splitter=configs.dataset.genre_splitter)
imax_count = imax.shape[0]
imax_percent = round(imax_count / mvl_dataset.movies.shape[0] * 100, 2)

print("IMAX", imax_count, f"({imax_percent}%)","- min genres count", imax.genres_count.min())
print("HEAD")
display(imax.head())

IMAX 158 (1.62%) - min genres count 2
HEAD


,movieId,title,genres,year,genres_count
123,150,Apollo 13 (1995),Adventure|Drama|IMAX,1995,3
322,364,"Lion King, The (1994)",Adventure|Animation|Children|Drama|Musical|IMAX,1994,6
512,595,Beauty and the Beast (1991),Animation|Children|Fantasy|Musical|Romance|IMAX,1991,6
1328,1797,Everest (1998),Documentary|IMAX,1998,2
2381,3159,Fantasia 2000 (1999),Animation|Children|Musical|IMAX,1999,4


<font class="answer">
IMAX n'est jamais utilisé seul ce qui confime qu'il est plus un "attribut" qu'un genre; une façon pratique d'encoder l'information dans le présent format.  Je le considère comme une modalité aberrante étant donné sa fréquence. Noter que la section exploration investiguera les autres modalités.

In [13]:
#
# enlever la modalite IMAX de genres
#
def remove_imax(genres_str, splitter):
    if "IMAX" in genres_str:
        genres_list = genres_str_to_array(genres_str, splitter)
        genres_list.remove("IMAX")
        genres_str = splitter.join(genres_list)
    
    return genres_str

no_imax = mvl_dataset.movies.genres[imax_crit].apply(remove_imax, 
                                                     splitter=configs.dataset.genre_splitter)
mvl_dataset.movies.loc[imax_crit, "genres"] = no_imax

# validation
print("Validation retrait IMAX")
display(mvl_dataset.movies[imax_crit].head())

Validation retrait IMAX


,movieId,title,genres,year
123,150,Apollo 13 (1995),Adventure|Drama,1995
322,364,"Lion King, The (1994)",Adventure|Animation|Children|Drama|Musical,1994
512,595,Beauty and the Beast (1991),Animation|Children|Fantasy|Musical|Romance,1991
1328,1797,Everest (1998),Documentary,1998
2381,3159,Fantasia 2000 (1999),Animation|Children|Musical,1999


In [14]:
#
# examiner (no genres listed)
#    
def show_no_genres():
    no_genres_crit = mvl_dataset.movies.genres.str.contains("(no genres listed)", regex=False)
    no_genres = mvl_dataset.movies[no_genres_crit]
    print("(no genres listed)", no_genres.shape[0])
    if no_genres.shape[0] > 0:
        display(no_genres.head())
    return no_genres.index
    
no_genres_index = show_no_genres()

(no genres listed) 34


,movieId,title,genres,year
8517,114335,La cravate (1957),(no genres listed),1957
8684,122888,Ben-hur (2016),(no genres listed),2016
8687,122896,Pirates of the Caribbean: Dead Men Tell No Tal...,(no genres listed),2017
8782,129250,Superfast! (2015),(no genres listed),2015
8836,132084,Let It Be Me (1995),(no genres listed),1995


<font class="answer">
Je vais utiliser le web scrapping pour tenter de remplacer (no genres listed).

In [15]:
def imdb_scap_genres(response, index, imdbId):
    success = False
    if response.ok:
        bs = BeautifulSoup(response.text, "html.parser")

        imdb_info = bs.find("script", attrs={"type": "application/ld+json"})
        imdb_json = json.loads(imdb_info.text)
                
        genres = scrap.get_nested_property(imdb_json, ["genre"])
        if not success and genres:
            success = True
        else:
            error = json.dumps(imdb_json, indent=4)
    else:
        error = response.reason

    if success:
        return (index, genres, response.url, None)
    else:
        
        return (index, None, response.url, error)

def imdb_scap_genres_apply_results(final_results, genres_splitter):
    for index, genres, url, error in final_results:
        if genres is None:
            if False:
                print()
                print(error)
                print()

            print(url, mvl_dataset.movies.title[index], "Failed")
        else:
            mvl_dataset.movies.loc[index, "genres"] = genres_splitter.join(genres)


# corriger genres via web scrapping avec imdb
genres_imdbIds = mvl_dataset.links.imdbId[no_genres_index]

if genres_imdbIds.shape[0] > 0:
    with hlp.Profile() as genres_profile:
        if True:
            results = scrap.imdb_requests_parallel(genres_imdbIds, 
                                                   configs.web_scraping, 
                                                   imdb_scap_genres,
                                                   executor=configs.executor)
            imdb_scap_genres_apply_results(results, configs.dataset.genre_splitter)
        else:
            # les resultats ne sont pas appliques ici
            # ce code n'existe que pour mesurer le gain du multithreading
            scrap.imdb_requests(genres_imdbIds, 
                                configs.web_scraping, 
                                imdb_scap_genres)
    print(f"Web scraping genres: {genres_profile.round_duration(2)}s")
        
show_unique_genres()
show_no_genres();

  0%|          | 0/34 [00:00<?, ?it/s]

Web scraping genres: 4.34s
Genres: 23 modalités


{'Action',
 'Adventure',
 'Animation',
 'Biography',
 'Children',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'Film-Noir',
 'History',
 'Horror',
 'Music',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Short',
 'Thriller',
 'War',
 'Western'}

(no genres listed) 0


<font class="answer">

Afin de simplifier la partie exploration, je vais ajouter *imdbId* à *movies.csv* fin d'avoir l'information dans une seule base de données.

In [16]:
#
# ajout imdbId a movies
#
mvl_dataset.movies = pd.merge(mvl_dataset.movies, 
                              mvl_dataset.links[["movieId", "imdbId"]],
                              how="left", 
                              on="movieId")

# pour faciliter la visualization, mettre immediatement imdbId
mvl_dataset.movies.insert(1, 'imdbId', mvl_dataset.movies.pop('imdbId'))

# validation
print("Validation ajout imdbId")
display(mvl_dataset.movies.head())

Validation ajout imdbId


,movieId,imdbId,title,genres,year
0,1,0114709,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,0113497,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,0113228,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,0114885,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
4,5,0113041,Father of the Bride Part II (1995),Comedy,1995


## ratings.csv

<font class="answer">
    
Mettre description ici?

In [17]:
print("Ratings", mvl_dataset.ratings.shape)
print("Head")
display(mvl_dataset.ratings.head())
clstr.show_na(mvl_dataset.ratings)
clstr.show_types(mvl_dataset.ratings)

Ratings (100836, 4)
Head


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


Valeur manquante 0 (0.0%)
Types


,userId,movieId,rating,timestamp
Type,int64,int64,float64,int64


<font class="answer">
  
*rating*, bien que montré en nombre flottant, est en fait un système d'étoiles ou la mesure de base est$\frac{1}{2}$. Je garde en quantitatif pour le moment car il semble avantageux pour faire un lien avec les données dans *movies.csv*. L'exploration pourra décider s'il est pertinant de passer en catégoriel si on traite les *ratings.csv* à part (système de recommendation collaboratif).
    
*timestamp* est en fait une date dans le format UTC. Ceci semble suggérer qu'un utilisateur peut avoir voté plus d'une fois pour le même film.

In [18]:
#
# verification nombre de votes par couple (utilisateur, film)
#
votes = mvl_dataset.ratings.groupby(by=["userId", "movieId"]).size().max()
print("Nombre de votes par (userId, movieId):", votes)

Nombre de votes par (userId, movieId): 1


<font class="answer">
Un seul vote par film. La variable timestamp semble donc superflue dans cette base de donnée.

In [19]:
mvl_dataset.ratings.drop("timestamp", axis=1, inplace=True, errors="ignore")

<font class="answer">

Pour movies.csv, il apparait judicieux d'extraire quelques statistiques sur *rating*

In [20]:
#
# creer des variables pour movies
#
ratings_stats = mvl_dataset.ratings[["movieId", "rating"]] \
                           .groupby(by="movieId") \
                           .agg(["count", "min", "max", "mean", "median"])

def join_name(multiindex_name):
    if "" in multiindex_name:
        return multiindex_name[0]
    else:
        return "_".join(multiindex_name)

ratings_stats.columns = [join_name(a) for a in ratings_stats.columns.to_flat_index()]
ratings_stats.reset_index(inplace=True)
print("Rating stats")
display(ratings_stats.round(2))

Rating stats


,movieId,rating_count,rating_min,rating_max,rating_mean,rating_median
0,1,215,0.5,5.0,3.92,4.0
1,2,110,0.5,5.0,3.43,3.5
2,3,52,0.5,5.0,3.26,3.0
3,4,7,1.0,3.0,2.36,3.0
4,5,49,0.5,5.0,3.07,3.0
...,...,...,...,...,...,...
9719,193581,1,4.0,4.0,4.00,4.0
9720,193583,1,3.5,3.5,3.50,3.5
9721,193585,1,3.5,3.5,3.50,3.5
9722,193587,1,3.5,3.5,3.50,3.5


In [21]:
#
# valider que ratings_stats contient bien les meme films que movies.csv
#
movies_ratings = pd.merge(mvl_dataset.movies, ratings_stats, how="left", on="movieId")
movies_ratings_na = clstr.show_na(movies_ratings)

Valeur manquante 18 (0.2%)


,movieId,imdbId,title,genres,year,rating_count,rating_min,rating_max,rating_mean,rating_median
816,1076,0055018,"Innocents, The (1961)",Drama|Horror|Thriller,1961,NaN,NaN,NaN,NaN,NaN
2211,2939,0046126,Niagara (1953),Drama|Thriller,1953,NaN,NaN,NaN,NaN,NaN
2499,3338,0097372,For All Mankind (1989),Documentary,1989,NaN,NaN,NaN,NaN,NaN
2587,3456,0191043,"Color of Paradise, The (Rang-e khoda) (1999)",Drama,1999,NaN,NaN,NaN,NaN,NaN
3118,4194,0037800,I Know Where I'm Going! (1945),Drama|Romance|War,1945,NaN,NaN,NaN,NaN,NaN
4037,5721,0082175,"Chosen, The (1981)",Drama,1981,NaN,NaN,NaN,NaN,NaN
4506,6668,0235060,"Road Home, The (Wo de fu qin mu qin) (1999)",Drama|Romance,1999,NaN,NaN,NaN,NaN,NaN
4598,6849,0066344,Scrooge (1970),Drama|Fantasy|Musical,1970,NaN,NaN,NaN,NaN,NaN
4704,7020,0102721,Proof (1991),Comedy|Drama|Romance,1991,NaN,NaN,NaN,NaN,NaN
5020,7792,0071970,"Parallax View, The (1974)",Thriller,1974,NaN,NaN,NaN,NaN,NaN


<font class="answer">
    
Tous les films n'ont pas reçu de votes. Je pourrais utiliser le web scraping pour remplir rating_mean mais l'echelle sur IMDB n'est pas la même que MovieLens. De plus les autres statistiques ne pourraient pas être extraite. Comme il n'y a que très peux d'éléments manquants je vais les ignorer.

In [22]:
#
# enlever valeurs manquantes et valider shape resultante
#
print("Avant retrait:", movies_ratings.shape)
movies_ratings.drop(index=movies_ratings_na, axis=0, inplace=True)
print("Après retrait:", movies_ratings.shape)

#
# valider les types (drop peux changer les choses du aux NA)
#
clstr.show_types(movies_ratings)

Avant retrait: (9742, 10)
Après retrait: (9724, 10)
Types


,movieId,imdbId,title,genres,year,rating_count,rating_min,rating_max,rating_mean,rating_median
Type,int64,string[python],object,object,int64,float64,float64,float64,float64,float64


In [23]:
#
# rating_count est en fait un entier, changer son type
#
movies_ratings.rating_count = movies_ratings.rating_count.astype(np.int64)

In [24]:
#
# sauvegarde des traitements
#
csv_path = mvl_dataset.movies_path.split(".")[0]
csv_path = "".join([csv_path, "_pretraitement", ".csv"])

print("Sauvegarde", csv_path)
movies_ratings.to_csv(csv_path, index=False)

Sauvegarde dataset/movies_pretraitement.csv
